In [1]:
import pandas as pd
import numpy as np
import os
import joblib
import logging
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


os.makedirs('logs', exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('' \
        'logs/train_metrics.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

data = pd.read_csv("/Users/lubovmoskalenko/Documents/python/-pabd25/data/processed/merged_cleaned.csv")
data = data.dropna(subset=['total_meters', 'price'])

X = data[['total_meters']]
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = np.mean(np.abs(y_test - y_pred))

logging.info(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
logging.info(f"Корень из MSE (RMSE): {rmse:.2f}")
logging.info(f"Коэффициент детерминации R²: {r2:.6f}")
logging.info(f"Средняя абсолютная ошибка (MAE): {mae:.2f} рублей")
logging.info(f"Коэффициент при площади: {model.coef_[0]:.2f}")
logging.info(f"Свободный член (intercept): {model.intercept_:.2f}")

os.makedirs('models', exist_ok=True)
model_path = 'models/linear_regression_model.pkl'

joblib.dump(model, model_path)

loaded_model = joblib.load(model_path)

2025-05-21 21:46:04,064 - INFO - Среднеквадратичная ошибка (MSE): 252117308792571.66
2025-05-21 21:46:04,066 - INFO - Корень из MSE (RMSE): 15878202.32
2025-05-21 21:46:04,066 - INFO - Коэффициент детерминации R²: 0.646965
2025-05-21 21:46:04,067 - INFO - Средняя абсолютная ошибка (MAE): 12051925.02 рублей
2025-05-21 21:46:04,067 - INFO - Коэффициент при площади: 825397.44
2025-05-21 21:46:04,067 - INFO - Свободный член (intercept): -15501852.48


In [2]:
import pandas as pd
import joblib


model = joblib.load('models/linear_regression_model.pkl')

try:
    user_input = float(input("Введите площадь квартиры в м²: "))
    if user_input <= 0:
        raise ValueError("Площадь должна быть положительным числом.")
    input_df = pd.DataFrame({'total_meters': [user_input]})
    predicted_price = model.predict(input_df)[0]
    print(f"Предсказанная цена квартиры площадью {user_input} м²: {predicted_price:,.0f} ₽")

except ValueError as e:
    print(f"Ошибка ввода: {e}")

Предсказанная цена квартиры площадью 42.0 м²: 19,164,840 ₽
